# EXAMPLE DATA

In [1]:
P = 50          # underlying price
K = 50          # strike price
T = 1           # years to maturity
sigma = 0.4     # volatility of underlying
r = 0.02        # risk-free rate
q = 0.03        # dividend yield

# FUNCTIONS

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm

def call(S, K):
    return np.maximum(S - K, 0)

def put(S, K):
    return np.maximum(K - S, 0)

def callBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-q * T) * s * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)

def putBS(S, K, T, sigma, r, q):
    def f(s):
        s = s if s != 0 else 1.0e-6
        d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return np.exp(-r * T) * K * norm.cdf(-d2) - np.exp(-q * T) * s * norm.cdf(-d1)

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)

def callData(P, K, T, sigma, r, q):
    # the black scholes value as the function of the underlying price
    S = [val for val in np.arange(0, 100.1, 0.1)]

    bs_values = [callBS(s, K, T, sigma, r, q) for s in S]

    def leverage_portfolio(S, delta):
        return (delta * S) - ((delta * P) - callBS(P, K, T, sigma, r, q))

    # delta for call
    d1 = (np.log(P / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    delta_1 = np.exp(-q*T) * norm.cdf(d1)

    leverage_values = [leverage_portfolio(s, delta_1) for s in S]

    tang_leverage_value = leverage_portfolio(P, delta_1)

    return S, bs_values, leverage_values, delta_1, tang_leverage_value


def putData(P, K, T, sigma, r, q):
    # the black scholes value as the function of the underlying price
    S = [val for val in np.arange(0, 100.1, 0.1)]
    bs_values = [putBS(s, K, T, sigma, r, q) for s in S]

    def leverage_portfolio(S, delta):
        return (delta * S) - ((delta * P) - putBS(P, K, T, sigma, r, q))

    # delta for call
    d1 = (np.log(P / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    delta_1 = - np.exp(-q*T) * norm.cdf(-d1)

    leverage_values = [leverage_portfolio(s, delta_1) for s in S]

    tang_leverage_value = leverage_portfolio(P, delta_1)

    return S, bs_values, leverage_values, delta_1, tang_leverage_value


# CALCULATIONS

In [4]:
S, bs_values_call, leverage_values_c, delta_call, tang_leverage_value = callData(P, K, T, sigma, r, q)
S, bs_values_put,  leverage_values_p, delta_put,  tang_leverage_value = putData(P, K, T, sigma, r, q)

cash_call = callBS(P, K, T, sigma, r, q) - delta_call*P
cash_put = putBS(P, K, T, sigma, r, q) - delta_put*P

print(f'The delta for the call is {delta_call:.3f} when the underlying price is ${P:.2f}')
print(f'The call delta-hedge portfolio holds {delta_call:.3f} shares of the underlying with a cash position of {cash_call:.2f}')

print(f'The delta for the put is {delta_put:.3f} when the underlying price is ${P:.2f}')
print(f'The put delta-hedge portfolio holds {delta_put:.3f} shares of the underlying with a cash position of {cash_put:.2f}')

The delta for the call is 0.553 when the underlying price is $50.00
The call delta-hedge portfolio holds 0.553 shares of the underlying with a cash position of -20.14
The delta for the put is -0.418 when the underlying price is $50.00
The put delta-hedge portfolio holds -0.418 shares of the underlying with a cash position of 28.87


# FIGURES

In [5]:
import plotly.graph_objects as go

name = 'Call Value'
text = 'European Call'
bs_values = bs_values_call
leverage_values = leverage_values_c

fig = go.Figure()
fig.add_trace(go.Scatter(x=S, y=bs_values, mode="lines", name=name))
fig.add_trace(go.Scatter(x=S, y=leverage_values, mode="lines", name="Delta Hedge")        )
fig.update_layout(hovermode="x unified")
fig.update_yaxes(title=None)
fig.update_layout(yaxis_tickformat=",.2f", xaxis_tickformat=",.2f")
fig.update_xaxes(title="Underlying Price")
fig.update_layout(
    title={
        "text": text,
        "y": 0.96,
        "x": 0.2,
        "xanchor": "center",
        "yanchor": "bottom",
    }
)
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig.show()

In [6]:
import plotly.graph_objects as go

name = 'Put Value'
text = 'European Put'
bs_values = bs_values_put
leverage_values = leverage_values_p

fig = go.Figure()
fig.add_trace(go.Scatter(x=S, y=bs_values, mode="lines", name=name))
fig.add_trace(go.Scatter(x=S, y=leverage_values, mode="lines", name="Delta Hedge")        )
fig.update_layout(hovermode="x unified")
fig.update_yaxes(title=None)
fig.update_layout(yaxis_tickformat=",.2f", xaxis_tickformat=",.2f")
fig.update_xaxes(title="Underlying Price")
fig.update_layout(
    title={
        "text": text,
        "y": 0.96,
        "x": 0.2,
        "xanchor": "center",
        "yanchor": "bottom",
    }
)
fig.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))
fig.show()